# **Preprocessing and metrics evaluation**

In [9]:
# !pip install -q xgboost scikit-learn tensorflow==2.15.0   # uncomment if needed

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix, roc_auc_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras


DATA_PATH = "/content/Crop_recommendation.csv"  # change if needed


# 1) Load dataset (robust target detection)
df = pd.read_csv(DATA_PATH)
print("Loaded CSV shape:", df.shape)
print("Columns:", df.columns.tolist())

# try to find a target column name automatically
for candidate in ['label','Label','crop','Crop','target','Target','y','Y','class','Class']:
    if candidate in df.columns:
        target_col = candidate
        break
else:
    target_col = df.columns[-1]  # fallback: last column
print("Using target column:", target_col)

# 2) Basic preprocessing: drop NA, separate X,y
df = df.dropna().reset_index(drop=True)
X = df.drop(columns=[target_col])
y = df[target_col].astype(str)   # ensure categorical/string for LabelEncoder

# handle any non-numeric features: simple one-hot or label encode
# If there are categorical/text columns, do one-hot for simplicity
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
if len(cat_cols):
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
print("Feature shape after encoding:", X.shape)

# 3) Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)
num_classes = len(le.classes_)
print("Classes ({}): {}".format(num_classes, le.classes_))

# 4) Train/test split
X_train, X_test, y_train, y_test = train_test_split(X.values, y_enc, test_size=0.20, random_state=42, stratify=y_enc)
print("Train/Test sizes:", X_train.shape, X_test.shape)

# 5) Scale features (important for SGD)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Utility: evaluation printing
def print_metrics(name, y_true, y_pred, y_proba=None):
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    print(f"\n=== {name} ===")
    print("Accuracy: {:.4f}".format(acc))
    print("Precision (weighted): {:.4f}".format(prec))
    print("Recall (weighted): {:.4f}".format(rec))
    print("F1-score (weighted): {:.4f}".format(f1))
    print("Classification report:\n", classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
    print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
    if (y_proba is not None) and (num_classes == 2):
        try:
            auc = roc_auc_score(y_true, y_proba[:,1])
            print("ROC AUC: {:.4f}".format(auc))
        except Exception:
            pass


Loaded CSV shape: (2200, 8)
Columns: ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label']
Using target column: label
Feature shape after encoding: (2200, 7)
Classes (22): ['apple' 'banana' 'blackgram' 'chickpea' 'coconut' 'coffee' 'cotton'
 'grapes' 'jute' 'kidneybeans' 'lentil' 'maize' 'mango' 'mothbeans'
 'mungbean' 'muskmelon' 'orange' 'papaya' 'pigeonpeas' 'pomegranate'
 'rice' 'watermelon']
Train/Test sizes: (1760, 7) (440, 7)


# **MODEL 1: SGDClassifier **

In [10]:
sgd = SGDClassifier(loss='log_loss', penalty='elasticnet', max_iter=2000, tol=1e-3, random_state=42)
sgd.fit(X_train_scaled, y_train)
y_pred_sgd = sgd.predict(X_test_scaled)
y_proba_sgd = None
if hasattr(sgd, "predict_proba"):
    y_proba_sgd = sgd.predict_proba(X_test_scaled)
print_metrics("SGDClassifier (sklearn)", y_test, y_pred_sgd, y_proba_sgd)

# Save SGD model (sklearn)
import joblib
joblib.dump(sgd, "sgd_model.joblib")
joblib.dump(scaler, "scaler.joblib")
joblib.dump(le, "label_encoder.joblib")


=== SGDClassifier (sklearn) ===
Accuracy: 0.9705
Precision (weighted): 0.9745
Recall (weighted): 0.9705
F1-score (weighted): 0.9712
Classification report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        20
      banana       1.00      1.00      1.00        20
   blackgram       1.00      0.95      0.97        20
    chickpea       1.00      1.00      1.00        20
     coconut       1.00      1.00      1.00        20
      coffee       1.00      1.00      1.00        20
      cotton       1.00      1.00      1.00        20
      grapes       1.00      1.00      1.00        20
        jute       0.83      0.95      0.88        20
 kidneybeans       1.00      1.00      1.00        20
      lentil       1.00      0.95      0.97        20
       maize       0.77      1.00      0.87        20
       mango       1.00      1.00      1.00        20
   mothbeans       0.94      0.85      0.89        20
    mungbean       1.00      1.0

['label_encoder.joblib']

In [11]:
# ========== MODEL 2: XGBoost ==========
# Use scikit-learn API of XGBoost
xgb_clf = xgb.XGBClassifier(
    objective='multi:softprob' if num_classes>2 else 'binary:logistic',
    use_label_encoder=False,
    eval_metric='mlogloss' if num_classes>2 else 'logloss',
    n_estimators=200,
    max_depth=6,
    random_state=42,
    n_jobs=-1
)
xgb_clf.fit(X_train, y_train)   # note: XGBoost can take unscaled features
y_pred_xgb = xgb_clf.predict(X_test)
y_proba_xgb = xgb_clf.predict_proba(X_test)
print_metrics("XGBoost", y_test, y_pred_xgb, y_proba_xgb)
joblib.dump(xgb_clf, "xgb_model.joblib")




/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [08:25:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== XGBoost ===
Accuracy: 0.9932
Precision (weighted): 0.9935
Recall (weighted): 0.9932
F1-score (weighted): 0.9931
Classification report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        20
      banana       1.00      1.00      1.00        20
   blackgram       1.00      1.00      1.00        20
    chickpea       1.00      1.00      1.00        20
     coconut       1.00      1.00      1.00        20
      coffee       1.00      1.00      1.00        20
      cotton       1.00      1.00      1.00        20
      grapes       1.00      1.00      1.00        20
        jute       0.95      1.00      0.98        20
 kidneybeans       1.00      1.00      1.00        20
      lentil       1.00      0.90      0.95        20
       maize       1.00      1.00      1.00        20
       mango       1.00      1.00      1.00        20
   mothbeans       0.95      1.00      0.98        20
    mungbean       0.95      1.00      0.98     

['xgb_model.joblib']

In [12]:
# ========== MODEL 3: Soft Voting Ensemble (SGD + XGBoost) ==========
# VotingClassifier expects estimators that implement predict_proba for soft voting.
# sklearn.SGDClassifier with loss='log' supports predict_proba. XGB supports predict_proba.
voting = VotingClassifier(estimators=[('sgd', sgd), ('xgb', xgb_clf)], voting='soft', n_jobs=-1)
voting.fit(X_train_scaled, y_train)   # both need scaled input; xgb was trained on raw earlier, but here we train voting on scaled
y_pred_voting = voting.predict(X_test_scaled)
# get predict_proba if available
if hasattr(voting, "predict_proba"):
    y_proba_voting = voting.predict_proba(X_test_scaled)
else:
    y_proba_voting = None
print_metrics("Soft Voting Ensemble (SGD + XGB)", y_test, y_pred_voting, y_proba_voting)
joblib.dump(voting, "voting_model.joblib")



=== Soft Voting Ensemble (SGD + XGB) ===
Accuracy: 0.9886
Precision (weighted): 0.9890
Recall (weighted): 0.9886
F1-score (weighted): 0.9886
Classification report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        20
      banana       1.00      1.00      1.00        20
   blackgram       1.00      1.00      1.00        20
    chickpea       1.00      1.00      1.00        20
     coconut       1.00      1.00      1.00        20
      coffee       1.00      1.00      1.00        20
      cotton       1.00      1.00      1.00        20
      grapes       1.00      1.00      1.00        20
        jute       0.95      0.95      0.95        20
 kidneybeans       1.00      1.00      1.00        20
      lentil       1.00      0.90      0.95        20
       maize       0.95      1.00      0.98        20
       mango       1.00      1.00      1.00        20
   mothbeans       0.95      1.00      0.98        20
    mungbean       0.95

['voting_model.joblib']

In [13]:
# ========== MODEL 4: Keras Neural Network (for TFLite conversion) ==========
# We'll build a small MLP. Input dims from X (scaled used).
input_dim = X_train_scaled.shape[1]

def make_keras_model(input_dim, num_classes):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(input_dim,)))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(64, activation='relu'))
    if num_classes == 2:
        model.add(keras.layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    else:
        model.add(keras.layers.Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

keras_model = make_keras_model(input_dim, num_classes)
# Use scaled features for NN
history = keras_model.fit(X_train_scaled, y_train, validation_split=0.1, epochs=40, batch_size=32, verbose=1)
# Evaluate
if num_classes == 2:
    y_proba_keras = keras_model.predict(X_test_scaled).reshape(-1)
    y_pred_keras = (y_proba_keras > 0.5).astype(int)
    y_proba_keras = np.vstack([1-y_proba_keras, y_proba_keras]).T
else:
    y_proba_keras = keras_model.predict(X_test_scaled)
    y_pred_keras = np.argmax(y_proba_keras, axis=1)
print_metrics("Keras NN", y_test, y_pred_keras, y_proba_keras)

# Save raw Keras model (SavedModel)
keras_model.export("keras_crop_model")

Epoch 1/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1995 - loss: 2.8880 - val_accuracy: 0.5398 - val_loss: 1.9809
Epoch 2/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5725 - loss: 1.8071 - val_accuracy: 0.8466 - val_loss: 0.9679
Epoch 3/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8133 - loss: 0.9333 - val_accuracy: 0.8750 - val_loss: 0.5425
Epoch 4/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8456 - loss: 0.5803 - val_accuracy: 0.9489 - val_loss: 0.3860
Epoch 5/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8965 - loss: 0.4142 - val_accuracy: 0.9091 - val_loss: 0.3054
Epoch 6/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9018 - loss: 0.3261 - val_accuracy: 0.9318 - val_loss: 0.2587
Epoch 7/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9166 - loss: 0.2883 - val_accuracy: 0.9375 - val_loss: 0.2135
Epoch 8/40
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9158 - loss: 0.2623 - val_accuracy: 0.9659 - val_loss:

In [14]:
# ========== Convert Keras -> TFLite ==========
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
# Optional: quantization (uncomment if desired)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Provide a representative_dataset_gen if doing post-training quantization (not added here)
tflite_model = converter.convert()
open("keras_crop_model.tflite", "wb").write(tflite_model)
print("Saved TFLite model to keras_crop_model.tflite")
# You can test the tflite model using the TFLite interpreter:
interpreter = tf.lite.Interpreter(model_path="keras_crop_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("TFLite input details:", input_details)
print("TFLite output details:", output_details)

# Example wrapper for TFLite inference
def run_tflite_inference(tflite_path, sample_array):
    inter = tf.lite.Interpreter(model_path=tflite_path)
    inter.allocate_tensors()
    inp = inter.get_input_details()[0]
    out = inter.get_output_details()[0]
    # ensure sample_array shape matches
    sample = np.array(sample_array, dtype=np.float32).reshape(1, -1)
    inter.set_tensor(inp['index'], sample)
    inter.invoke()
    pred = inter.get_tensor(out['index'])
    return pred


Saved artifact at '/tmp/tmp0k72j476'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7), dtype=tf.float32, name='keras_tensor_22')
Output Type:
  TensorSpec(shape=(None, 22), dtype=tf.float32, name=None)
Captures:
  136158834045200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136158787731088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136158787730896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136158787731280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136158787730512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136158787727440: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved TFLite model to keras_crop_model.tflite
TFLite input details: [{'name': 'serving_default_keras_tensor_22:0', 'index': 0, 'shape': array([1, 7], dtype=int32), 'shape_signature': array([-1,  7], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [15]:
# ========== Optional: Distill Ensemble -> Keras Student (so you can TFLite the student) ==========
# We'll predict soft probabilities with the voting ensemble and train a small Keras model to mimic them.
if hasattr(voting, "predict_proba"):
    soft_targets = voting.predict_proba(X_train_scaled)
    # build student model same architecture but smaller
    student = make_keras_model(input_dim, num_classes)
    # If multi-class, use categorical crossentropy on the soft labels -> train with probabilities (use from_logits=False)
    if num_classes == 2:
        # binary distillation: use probability of class 1
        student.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        teacher_labels = soft_targets[:,1]
    else:
        student.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        teacher_labels = soft_targets  # soft targets
    # For categorical_crossentropy with soft labels, use y as one-hot soft vectors
    if num_classes > 2:
        student.fit(X_train_scaled, teacher_labels, epochs=50, batch_size=64, verbose=1, validation_split=0.1)
        y_proba_student = student.predict(X_test_scaled)
        y_pred_student = np.argmax(y_proba_student, axis=1)
    else:
        student.fit(X_train_scaled, teacher_labels, epochs=50, batch_size=64, verbose=1, validation_split=0.1)
        y_proba_student = student.predict(X_test_scaled).reshape(-1)
        y_pred_student = (y_proba_student > 0.5).astype(int)
    print_metrics("Student (distilled from Ensemble)", y_test, y_pred_student, y_proba_student)
    # Save and convert student to TFLite
    student.export("student_model_savedmodel")
    converter_s = tf.lite.TFLiteConverter.from_saved_model("student_model_savedmodel")
    tflite_student = converter_s.convert()
    open("student_model.tflite", "wb").write(tflite_student)
    print("Saved distilled student TFLite:", "student_model.tflite")

print("\nAll done. Files saved in the Colab working directory:")
print(os.listdir('.'))

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.1451 - loss: 3.0057 - val_accuracy: 0.5057 - val_loss: 2.5574
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5219 - loss: 2.4184 - val_accuracy: 0.6420 - val_loss: 1.8761
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6645 - loss: 1.7727 - val_accuracy: 0.7727 - val_loss: 1.2819
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8000 - loss: 1.2493 - val_accuracy: 0.8977 - val_loss: 0.9322
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8488 - loss: 0.9474 - val_accuracy: 0.9205 - val_loss: 0.7675
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8578 - loss: 0.8134 - val_accuracy: 0.9375 - val_loss: 0.6753
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8895 - loss: 0.6995 - val_accuracy: 0.9489 - val_loss: 0.6181
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9038 - loss: 0.6447 - val_accuracy: 0.9432 - val_loss

In [20]:
# !pip install geocoder requests

import geocoder
import requests
import numpy as np
import joblib
import tensorflow as tf

# ====== Step 1: Detect Location (lat, lon) ======
lat , lon=19.994544, 74.379912
#g = geocoder.ip('me')
#if g.ok:
 #   lat, lon = g.latlng
  #  print(f"Detected location: lat={lat}, lon={lon}")
#else:
 #   lat, lon = 23.5, 85.3  # fallback: Ranchi, Jharkhand
  #  print("Fallback location:", lat, lon)

# ====== Step 2: Fetch Weather Data (OpenWeatherMap API) ======
OWM_API_KEY = "dba1658fffa3efcf30741487f4e00a4c"  # <-- replace with your key

def get_weather(lat, lon, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    r = requests.get(url).json()
    temp = r['main']['temp']
    humidity = r['main']['humidity']
    rainfall = r.get('rain', {}).get('1h', 0.0)  # default 0 if missing
    return temp, humidity, rainfall

temperature, humidity, rainfall = get_weather(lat, lon, OWM_API_KEY)
print(f"Weather: Temp={temperature}°C, Humidity={humidity}%, Rainfall={rainfall} mm")

# ====== Step 3: Fetch Soil Data (SoilGrids API) ======
def get_soil(lat, lon):
    url = f"https://rest.isric.org/soilgrids/v2.0/properties/query?lon={lon}&lat={lat}&property=nitrogen&property=phh2o&depth=0-5cm"
    r = requests.get(url).json()

    # Debug: check keys available
    # print(r.keys())

    def safe_extract(layer_idx, depth_idx=0):
        try:
            val = r['properties']['layers'][layer_idx]['depths'][depth_idx]['values']['mean']
            return float(val) if val is not None else None
        except Exception as e:
            return None

    N = safe_extract(0)  # nitrogen
    ph = safe_extract(1)  # phh2o

    # Convert / fill defaults
    N = N * 1000 if N is not None else 90.0   # default ~90 (mg/kg)
    ph = ph if ph is not None else 6.5

    # SoilGrids doesn’t give P, K → approximate or keep static defaults
    P, K = 40.0, 40.0

    return N, P, K, ph


# ====== Step 4: Build Feature Vector ======
N, P, K, ph = get_soil(lat, lon)
print(f"Soil Data -> N={N}, P={P}, K={K}, pH={ph}")

env_features = {
    "N": N,
    "P": P,
    "K": K,
    "temperature": temperature,
    "humidity": humidity,
    "ph": ph,
    "rainfall": rainfall
}
print("Final feature vector:", env_features)

feature_vector = [
    env_features["N"],
    env_features["P"],
    env_features["K"],
    env_features["temperature"],
    env_features["humidity"],
    env_features["ph"],
    env_features["rainfall"]
]
feature_vector = np.array(feature_vector, dtype=np.float32).reshape(1, -1)

# ====== Step 5: Load Preprocessing Objects ======
scaler = joblib.load("scaler.joblib")
le = joblib.load("label_encoder.joblib")
X_scaled = scaler.transform(feature_vector)

# ====== Step 6: Run Inference with TFLite Model ======
interpreter = tf.lite.Interpreter(model_path="student_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], X_scaled.astype(np.float32))
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])

# ====== Step 7: Decode Prediction ======
if prediction.shape[1] == 1:  # binary case
    pred_class = int(prediction[0][0] > 0.5)
else:
    pred_class = np.argmax(prediction, axis=1)[0]

recommended_crop = le.inverse_transform([pred_class])[0]
print("\n🌾 Recommended Crop:", recommended_crop)


Weather: Temp=28.27°C, Humidity=68%, Rainfall=0.0 mm
Soil Data -> N=143000.0, P=40.0, K=40.0, pH=72.0
Final feature vector: {'N': 143000.0, 'P': 40.0, 'K': 40.0, 'temperature': 28.27, 'humidity': 68, 'ph': 72.0, 'rainfall': 0.0}

🌾 Recommended Crop: cotton


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [21]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/Crop_recommendation.csv")

# List of all unique crops in the dataset
all_crops = df["label"].unique()

# Define rule-based crop rotation
rotation_rules = {
    # Cereals
    "rice": ["wheat", "mustard", "chickpea", "lentil"],
    "wheat": ["rice", "maize", "pulses", "groundnut"],
    "maize": ["soybean", "wheat", "potato", "pea"],
    "barley": ["rice", "maize", "pulses"],

    # Pulses
    "lentil": ["rice", "maize", "sorghum", "mustard"],
    "chickpea": ["rice", "maize", "sorghum"],
    "pigeonpeas": ["rice", "wheat", "mustard"],
    "mothbeans": ["sorghum", "pearl millet", "cowpea"],
    "mungbean": ["rice", "wheat", "mustard"],
    "blackgram": ["rice", "maize", "mustard"],
    "kidneybeans": ["rice", "maize", "vegetables"],

    # Oilseeds
    "groundnut": ["wheat", "mustard", "vegetables"],
    "soybean": ["wheat", "mustard", "vegetables"],
    "mustard": ["rice", "maize", "vegetables"],

    # Fibre
    "cotton": ["wheat", "mustard", "pulses", "vegetables"],
    "jute": ["wheat", "mustard", "pulses"],

    # Cash crops
    "sugarcane": ["wheat", "pulses", "vegetables"],
    "coffee": ["banana", "black pepper", "ginger"],

    # Fruits
    "banana": ["vegetables", "pulses", "ginger"],
    "mango": ["vegetables", "pulses", "groundnut"],
    "grapes": ["wheat", "mustard", "pulses"],
    "watermelon": ["maize", "pulses", "groundnut"],
    "muskmelon": ["maize", "pulses", "groundnut"],
    "apple": ["barley", "peas", "vegetables"],
    "orange": ["vegetables", "pulses", "ginger"],
    "papaya": ["vegetables", "legumes", "ginger"],
    "pomegranate": ["wheat", "vegetables", "pulses"],
    "coconut": ["banana", "vegetables", "ginger"]
}

# Fallback recommender
def recommend_next_crop(prev_crop):
    if prev_crop in rotation_rules:
        return rotation_rules[prev_crop]
    else:
        return [c for c in all_crops if c != prev_crop][:3]

# Generate recommendations for all crops in dataset
#for crop in all_crops:
    #print(f"After {crop} → {recommend_next_crop(crop)}")
print(f"After {recommended_crop} -> {recommend_next_crop(recommended_crop)}")

After cotton -> ['wheat', 'mustard', 'pulses', 'vegetables']
